In [1]:
] activate ../.

In [2]:
using AutomotiveDrivingModels
using AutoViz
using AutomotiveSensors
using AutomotivePOMDPs
using Reel
using Revise

using EmergencyBrakingSystem


┌ Info: Recompiling stale cache file /home/xubuntu/.julia/compiled/v1.0/EmergencyBrakingSystem/YXMuK.ji for EmergencyBrakingSystem [3f59e4f0-e69d-11e8-310d-15d6353296e9]
└ @ Base loading.jl:1187


In [3]:

# Definition ego vehicle and pedestrian behavior

scenarios = ["CPCN", "CPAN25", "CPAN75", "CPFA"]
vut_speeds = [10., 15., 20., 25., 30., 35., 40., 45., 50., 55., 60.]
vut_speeds = vut_speeds / 3.6
hit_points = [0., 10., 20., 30., 40., 50.]


ego_v = vut_speeds[9]
hit_point = hit_points[1]
scenario = scenarios[1]


### Simulate scenario with parameters above defined
# generate scenario based on scenario type
(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles, scenario_id) = generate_scenario(scenario, ego_v, hit_point)
# simulate scenario
(rec, timestep, env, sensor, sensor_observations, ego_vehicle, ego_a, collision, collision_rate, ttc, risk, emergency_brake_request, prediction_obstacle) = EmergencyBrakingSystem.evaluate_scenario(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles)
#evaluate result
(collision2, emergency_brake_intervention, dv_collision, v_mean, a_mean, a_jerk, a_min) = evaluateScenarioMetric(ego_vehicle, emergency_brake_request, ego_a, collision)

#println(ego_x, " ", ego_y, " ", ego_v, " ", ped_x, " ", ped_y, " ", ped_v, " ", ped_theta, " ", obstacles)

(true, true, 7.638888888888889, 13.214285714285731, -9.0, 10.0, -10.0)

In [4]:
# Visualize scenario
duration, fps, render_hist = EmergencyBrakingSystem.animate_record(rec, timestep, env, ego_vehicle, sensor, sensor_observations, risk, ttc, collision_rate, emergency_brake_request, prediction_obstacle, StaticCamera(VecE2(38.0,0.0),25.0))
film = roll(render_hist, fps = fps, duration = duration)
#write("file.mp4", film) # Write to a webm video

Frames{MIME{Symbol("image/png")}}("/tmp/tmp6Pkdhi", 0x0000000000000040, 20.0, nothing)

In [5]:
# Evaluate EuroNCAP scenarios

using CSV
using DataFrames


algorithm = "EmergancyBrakingSystem"


policy = "no"


log_filename = string("results_", algorithm, "_", policy, ".csv")

println(log_filename)

scenarios = ["CPCN", "CPAN25", "CPAN75", "CPFA"]
vut_speeds = [50.] #[10., 15., 20., 25., 30., 35., 40., 45., 50., 55., 60.]
vut_speeds = vut_speeds / 3.6
hit_points = [0., 10., 20., 30., 40., 50.]


println("Algorithm to evaluate: ", algorithm, " Policy: ", policy, " (if specified)")

# Evaluate all EuroNCAP scenarios
rec, timestep, env, ego_vehicle, sensor, sensor_observations, risk, ttc, collision_rate, emergency_brake_request, prediction_obstacle, collision, ego_a
results = Vector[]
for scenario in scenarios
    for hit_point in hit_points
        for vut_speed in vut_speeds
            ego_v = vut_speed
            (ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles, scenario_id) = generate_scenario(scenario, ego_v, hit_point)
            if algorithm == "EmergancyBrakingSystem"
                (rec, timestep, env, sensor, sensor_observations, ego_vehicle, ego_a, collision, collision_rate, ttc, risk, emergency_brake_request, prediction_obstacle) = EmergencyBrakingSystem.evaluate_scenario(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles)
            elseif algorithm == "PedestrianAvoidancePOMDP"

            elseif algorithm == "PedestrianAvoidancePOMDP_EmergencyBrakingSystem"
                
            else
                println("No valid algorithm defined!")
                return false
            end
            
            (collision, emergency_brake_intervention, dv_collision, v_mean, a_mean, a_jerk, a_min) = evaluateScenarioMetric(ego_vehicle, emergency_brake_request, ego_a, collision)
            println(collision, " ", emergency_brake_intervention, " ", dv_collision, " ", v_mean, " ", a_mean, " ", a_jerk, " ", a_min)
            result = [scenario_id, ego_v, hit_point, collision, emergency_brake_intervention, dv_collision, v_mean, a_mean, a_jerk, a_min  ]
            push!(results, result)
        end
    end 
end  


# store results from scneario evaluation in log file
df = DataFrame(results)
df = DataFrame(Matrix(df)')
rename!(df, :x1 => :scenario_id, :x2 => :ego_v, :x3 => :hit_point, :x4 => :collision, :x5 => :eb_intervention)
rename!(df, :x6 => :dv_collision, :x7 => :v_mean, :x8 => :a_mean, :x9 => :a_jerk, :x10 => :a_min)

CSV.write(log_filename, df);
df


results_EmergancyBrakingSystem_no.csv
Algorithm to evaluate: EmergancyBrakingSystem Policy: no (if specified)
true true 5.138888888888889 12.6578282828283 -9.25 10.0 -10.0
false true 0.0 11.236861861861879 -9.5 20.0 -10.0
false true 0.0 11.236861861861879 -9.5 20.0 -10.0
false true 0.0 11.236861861861879 -9.5 20.0 -10.0
false true 0.0 11.236861861861879 -9.5 20.0 -10.0
false true 0.0 11.236861861861879 -9.5 20.0 -10.0
false true 0.0 11.236861861861879 -9.5 20.0 -10.0
false true 0.0 11.236861861861879 -9.5 20.0 -10.0
false true 0.0 11.236861861861879 -9.5 20.0 -10.0
false true 0.0 11.200532724505345 -9.5 20.0 -10.0
false true 0.0 11.236861861861879 -9.5 20.0 -10.0
false true 0.0 11.200532724505345 -9.5 20.0 -10.0
false true 0.0 11.236861861861879 -9.5 20.0 -10.0
false true 0.0 11.200532724505345 -9.5 20.0 -10.0
false true 0.0 11.236861861861879 -9.5 20.0 -10.0
false true 0.0 11.200532724505345 -9.5 20.0 -10.0
false true 0.0 11.236861861861879 -9.5 20.0 -10.0
false true 0.0 11.2005327245

,scenario_id,ego_v,hit_point,collision,eb_intervention,dv_collision,v_mean,a_mean,a_jerk,a_min
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,13.8889,0.0,1.0,1.0,5.13889,12.6578,-9.25,10.0,-10.0
2,1.0,13.8889,10.0,0.0,1.0,0.0,11.2369,-9.5,20.0,-10.0
3,1.0,13.8889,20.0,0.0,1.0,0.0,11.2369,-9.5,20.0,-10.0
4,1.0,13.8889,30.0,0.0,1.0,0.0,11.2369,-9.5,20.0,-10.0
5,1.0,13.8889,40.0,0.0,1.0,0.0,11.2369,-9.5,20.0,-10.0
6,1.0,13.8889,50.0,0.0,1.0,0.0,11.2369,-9.5,20.0,-10.0
7,2.0,13.8889,0.0,0.0,1.0,0.0,11.2369,-9.5,20.0,-10.0
8,2.0,13.8889,10.0,0.0,1.0,0.0,11.2369,-9.5,20.0,-10.0
9,2.0,13.8889,20.0,0.0,1.0,0.0,11.2369,-9.5,20.0,-10.0
